## Australian rainfall prediction

Goal: **To predict the next-day rain based on other atmospheric features**
    
Dataset: This dataset comprises a decade of daily weather observations from multiple locations across Australia. 

Source: https://www.kaggle.com/datasets/arunavakrchakraborty/australia-weather-data

### Data Description:
    
**Location** - Name of the city from Australia.

**MinTemp/MaxTemp** - The minimum/maximum temperature during a particular day. (degree Celsius)

**Rainfall** - Rainfall during a particular day. (millimeters)

**Evaporation** - Evaporation during a particular day. (millimeters)

**Sunshine** - Bright sunshine during a particular day. (hours)

**WindGusDir** - The direction of the strongest gust during a particular day. (16 compass points)

**WindGuSpeed** - Speed of strongest gust during a particular day. (kilometers per hour)

**WindDir9am / WindDir3pm** - The direction of the wind for 10 min prior to 9 am. / 3pm. (compass points)

**WindSpeed9am / WindSpeed3pm** - Speed of the wind for 10 min prior to 9 am. / 3pm. (kilometers per hour)

**Humidity9am / Humidity3pm** - The humidity of the wind at 9 am. / 3pm. (percent)

**Pressure9am / Pressure3pm** - Atmospheric pressure at 9 am. / 3pm. (hectopascals)

**Cloud9am / Cloud3pm** - Cloud-obscured portions of the sky at 9 am. / 3pm.(eighths)

**Temp9am / Temp3pm** - The temperature at 9 am. / 3pm.(degree Celsius)

**RainToday** - If today is rainy then ‘Yes’. If today is not rainy then ‘No’.

**RainTomorrow** - If tomorrow is rainy then 1 (Yes). If tomorrow is not rainy then 0 (No).

<div class="alert alert-block alert-success">The variables below were not presented in original dataset. However, I thought it would be wise to add regions and coordinates in order to plot the cities on the map (which I prepared in Tableau). Always better to see where the places  we are taking about are.

**State/Province** - State/Province of the locations in Australia

**Longitute/Latitude** - Coordinates of mentioned cities
</div>

It's a pity that there is no information about the date, however when we plot some variables I'm sure we will receive some seasonal trends in the data for example in min max temperature. 

Why do we measure "clouds", "wind direction","humidity","temperature" at 9am and 3pm? 

It is also important to become acquainted with Australian climate especially in respect of rainfall before we will immerse in the analysis of our dataset. Here are some key aspects:
    
1. Australia is located in the Southern Hemisphere and is surrounded by oceans (Southern Ocean,Pacific Ocean, Indian Ocean) and seas (Timor Sea,Arafura Sea,Coral Sea, Tasman Sea).
2. Dry and arid regions particularly in the central and western regions. These areas experience hot and dry conditions for much of the year, with limited rainfall. They are characterized by vast deserts, such as the Simpson Desert and the Great Victoria Desert.
3. Northern parts of Australia have tropical / subtropical climate - wet/dry seasons; monsoon seasons (wet) - from November to April,heavy rainfall,tropical cyclones; dry seasons - from May to October, lower humidity, clear skies
4. Southern / Southeastern regions have temparate climate - mild winters, moderate summers, rainfall throughout the year
5. Southern / Southwestern parts have mediterranean climate - mild, wet winters and hot,dry summers
6. Apline climate in southeastern regions (particularly the Australian Alps) - cold winters,snowfall,cool summers, higher rainfall due to orographic effects(rainfall caused by the lifting of moist air over mountains)
7. Coastale areas have mild temperatures and high humidity
8. Australia is prone to natural disasters, including tropical cyclones in the north and bushfires in various parts of the country.

To conclude: Australia's climate is highly diverse and can be broadly classified into several distinct regions based on precipitation patterns

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from scipy.stats import chi2_contingency
# from scipy.stats import kruskal

In [ ]:
raw_data=pd.read_csv("Weather Data.csv")
raw_data.head()

In [ ]:
weather_data = raw_data.copy()
weather_data = weather_data.drop(['row ID'],axis=1)
weather_data.info()
# 25 columns, 99516 rows
#there are some nulls 
#types: object, float64,int64

In [ ]:
weather_data.shape

In [ ]:
pd.set_option('display.max_columns', None) #to see all the columns; None - meaning there will be no limit to the number of columns displayed
weather_data.describe(include="all")

In [ ]:
#Missing values
NA_data = weather_data.isnull().sum()
NA_data

In [ ]:
NA_percentage = round(NA_data / weather_data.shape[0] *100,1)

In [ ]:
print("Percentage of missing values in each column: \n{}".format(NA_percentage))

In [ ]:
#Insights: 
#1. Are the missing values clustered in specific regions? 
#2. Better to start from numeric or categorical features? 

## Dealing with missing values
Common strategies: 

    1. remove rows/columns with NaNs - however this can lead to a loss of valuable information
    2. use statistical methods - for time-series data -> interpolation,extrapolation
    3. instead of filling in missing values, we can create an additional binary column to indicate whether a value is missing or not
    4. Imputation - replace missing values with estimated/predicted values

There are several methods to fill gaps in a categorical variable:

1. Mode Imputation: Replace the missing values with the most frequent category (mode) in the variable. 

2. Random Imputation: Replace the missing values with random samples from the distribution of the existing categories in the variable. 

3. Backward Fill (bfill) and Forward Fill (ffill): Fill the missing values with the previous (bfill) or next (ffill) non-missing value in the variable - when the data follows a temporal or sequential pattern.

4. K-Nearest Neighbors (KNN) Imputation: based on the similarity of other data points. 

5. Hot Deck Imputation: Replace missing values with values randomly selected from similar records based on certain criteria, such as matching values of other features.

6. Imputation Using Machine Learning Models like decision trees or random forests to predict the missing categories based on other features.


# Categorical variables

## <span style='color: #3F2E3E;'>Location - nominal</span>
#### <span style='color: #A78295;'>Name of the city from Australia </span>

In [ ]:
weather_data['Location'].value_counts()
#Why are there the most observations from Canberra,Sydney,Perth,Hobart..? Why this kind of hirarchy? How did they get this proportions?
#Well.. Firstly, these cities are the biggest, the most populous. Secondly, I noticed that the first listed cities are located in different regions on the map. 
#We distinguish 9 states/provinces, and the first 7 cities belongs to the 7 different regions. Circumstance? 

In [ ]:
#type(weather_data['Location'].value_counts()) #pandas.core.series.Series
#weather_data['Location'].value_counts().index
#weather_data['Location'].value_counts().index[:10]
#weather_data['Location'].value_counts()[:10]

location_counts = weather_data['Location'].value_counts()
first_names = location_counts.index[:10]
first_counts = location_counts[:10]

sns.barplot(x=first_names, y=first_counts,palette='viridis')

plt.xlabel('Location')
plt.ylabel('Count')
plt.title('Top 10 Locations by count')

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

sns.despine()  # Remove the top and right spines

# Add annotations for each bar
for index, value in enumerate(first_counts):
    plt.text(index, value, str(value), ha='center', va='bottom')
    
# e.g.if first_counts is [10, 25, 15, 30, 20], the enumerate() function will yield the following tuples: (0, 10), (1, 25), (2, 15), (3, 30), and (4, 20).
# index: The x-coordinate at which the text will be placed
# value: The y-coordinate at which the text will be placed
# str(value): The text that will be displayed at the specified coordinates
# ha - horizontal alignment
# va - vertical alignment
plt.show()

## <span style='color: #3F2E3E;'>State/Province - nominal</span>
#### <span style='color: #A78295;'>State or province of the cities in Australia </span>

In [ ]:
weather_data['State/Province'].value_counts()

In [ ]:
state_counts = weather_data['State/Province'].value_counts()

plt.figure(figsize=(8, 6))  #specifies the width and height of the figure in inches
sns.set_palette('tab20b')  

plt.pie(state_counts, labels=state_counts.index, autopct='%1.1f%%', startangle=140)

# autopct='%1.1f%%' - automatic percentage -> to display the percentage of each wedge(category)
# f - float, 1.1 - one digit after decimal point, %% - escape sequence to display % symbol
# By default, the first wedge starts from the positive x-axis (0 degrees) and proceeds counterclockwise - startangle

plt.title('State/Province Distribution')

plt.axis('equal')  # Equal aspect ratio ensures the pie is circular
plt.show()

## <span style='color: #3F2E3E;'>WindGustDir - nominal</span>
#### <span style='color: #A78295;'>The direction of the strongest gust during a particular day. (16 compass points)</span>

In [ ]:
pd.set_option('display.max_rows', None)
weather_data[weather_data['WindGustDir'].isnull()].head(100)
#In rows with missing WindGustDirection there are also NaN values in other variables
#NaN values in WindGustDir represent 6.6% of total number of rows, this is not so much, so that I decided to remove rows 
#with missing values in WindGustDir

In [ ]:
weather_data_cleaned_gust_dir = weather_data.copy()
weather_data_cleaned_gust_dir = weather_data_cleaned_gust_dir.dropna(subset=['WindGustDir'])
NA_data_cleaned_gust_dir = weather_data_cleaned_gust_dir.isnull().sum()
NA_data_cleaned_gust_dir

In [ ]:
#we removed the following number of missing observations:
NA_data - NA_data_cleaned_gust_dir

In [ ]:
weather_data_cleaned_gust_dir['WindGustDir'].value_counts()

In [ ]:
custom_order = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW']
reversed_custom_order = custom_order[::-1]
wind_gust_dir = weather_data_cleaned_gust_dir['WindGustDir'].value_counts().reindex(reversed_custom_order)
wind_gust_dir

In [ ]:
plt.figure(figsize=(8, 6))  #specifies the width and height of the figure in inches
sns.set_palette('tab20b')  

plt.pie(wind_gust_dir, labels=wind_gust_dir.index, autopct='%1.1f%%', startangle=100)

plt.title('The direction of the strongest gust during a particular day - frequency',pad=20) # pad=20 to make space between the title and the pie chart

plt.axis('equal')  # Equal aspect ratio ensures the pie is circular
plt.show()

## <span style='color: #3F2E3E;'>WindDir9am / WindDir3pm - nominal</span>
#### <span style='color: #A78295;'>The direction of the wind for 10 min prior to 9 am. / 3pm. (compass points)</span>

In [ ]:
# After removing NaN rows in WindGustDir, there is NaNs rows in: 
# WindDir9am         5265
# WindDir3pm          668
weather_data_cleaned_gust_dir.dropna(subset=['WindDir9am']).isnull().sum()
#I want to check if I remove missing values from WindDir9am, the missing values from WindDir3pm will disappear too. But only around 200 records contained NANs in both variables.  

In [ ]:
NA_percentage = round(NA_data_cleaned_gust_dir / weather_data_cleaned_gust_dir.shape[0] *100,1)
print("Percentage of missing values in each column: \n{}".format(NA_percentage))

In [ ]:
# Before filling null values
weather_data_dir = weather_data_cleaned_gust_dir.copy()
wind_dir_9am = weather_data_dir['WindDir9am'].value_counts().reindex(custom_order)
wind_dir_3pm = weather_data_dir['WindDir3pm'].value_counts().reindex(custom_order)
wind_dir = pd.DataFrame({'Direction': custom_order,'10 min prior to 9 am': wind_dir_9am.values, '10 min prior to 3 pm':wind_dir_3pm.values})
wind_dir.set_index('Direction', inplace=True)
ax = wind_dir.plot(kind='bar', figsize=(10, 6), width=0.5,color=['#3F4E4F', '#DCD7C9'], edgecolor='black')
ax.set_ylabel('Values')
ax.set_xlabel('Wind Direction')
ax.set_title('Wind Direction Comparison (9am vs. 3pm)')
plt.legend(title='Time', fontsize=10)
plt.xticks(rotation=45)
#plt.grid(axis='y', linestyle='--', alpha=0.7)
sns.despine()
plt.show()

In [ ]:
# After filling null values

In [ ]:
#A convenient solution in this case may be to replace NaNs with the most frequent category (mode) since we have only few missing values
#type(weather_data_cleaned['WindDir9am'].mode()) pandas.core.series.Series
weather_data_dir_with_mode = weather_data_dir.copy()
mode_winddir9am = weather_data_dir_with_mode['WindDir9am'].mode().iloc[0]
mode_winddir3pm = weather_data_dir_with_mode['WindDir3pm'].mode().iloc[0]
print(f"The mode in WindDri9am is: {mode_winddir9am}, while in WindDir3pm: {mode_winddir3pm}")

In [ ]:
weather_data_dir_with_mode['WindDir9am'] = weather_data_dir_with_mode['WindDir9am'].fillna(mode_winddir9am)
weather_data_dir_with_mode['WindDir3pm'] = weather_data_dir_with_mode['WindDir3pm'].fillna(mode_winddir3pm)

In [ ]:
weather_data_dir_with_mode.isnull().sum()

In [ ]:
wind_dir_9am = weather_data_dir_with_mode['WindDir9am'].value_counts().reindex(custom_order)
wind_dir_3pm = weather_data_dir_with_mode['WindDir3pm'].value_counts().reindex(custom_order)

In [ ]:
#doubled_custom_order = [dir for dir in custom_order for _ in range(2)]

In [ ]:
wind_dir = pd.DataFrame({'Direction': custom_order,'10 min prior to 9 am': wind_dir_9am.values, '10 min prior to 3 pm':wind_dir_3pm.values})
wind_dir

In [ ]:
wind_dir.set_index('Direction', inplace=True)
wind_dir

In [ ]:
# Plot the grouped barplot
ax = wind_dir.plot(kind='bar', figsize=(10, 6), width=0.5,color=['#3F4E4F', '#DCD7C9'], edgecolor='black')
ax.set_ylabel('Values')
ax.set_xlabel('Wind Direction')
ax.set_title('Wind Direction Comparison (9am vs. 3pm)')
plt.legend(title='Time', fontsize=10)
plt.xticks(rotation=45)
#plt.grid(axis='y', linestyle='--', alpha=0.7)
sns.despine()
plt.show()

## <span style='color: #3F2E3E;'>Cloud9am / Cloud3pm - ordinal</span>
#### <span style='color: #A78295;'>Cloud-obscured portions of the sky at 9 am. / 3pm.(scale in oktas - eighths)</span>

In meteorology, cloud cover is often measured in "oktas." An okta is a unit of measurement used to estimate the fraction of the sky covered by clouds at any given time. It is divided into 8 equal parts, with each okta representing 1/8th of the sky covered by clouds.However, we can distinguish also 0 oktas and 9 oktas. 

Here's how cloud cover is typically described in terms of oktas:

0 oktas: Completely clear sky, no clouds.

1 okta: Very few clouds, almost clear sky.

2 oktas: Partly cloudy, 25% of the sky covered by clouds.

3 oktas: Mostly cloudy.

4 oktas: More clouds, about half of the sky covered.

5 oktas: Overcast sky.

6 oktas: Cloudy, with about 75% of the sky covered by clouds.

7 oktas: Mostly covered.

8 oktas: Completely overcast, the entire sky covered by clouds.

9 oktas: Sky obscured by thick clouds.


In [ ]:
weather_data_clouds = weather_data_dir_with_mode.copy()
#weather_data_clouds.info()

In [ ]:
weather_data_clouds[['Cloud9am','Cloud3pm']].head(20)

In [ ]:
sorted(weather_data_clouds['Cloud3pm'].unique())

In [ ]:
sorted(weather_data_clouds['Cloud9am'].unique())

In [ ]:
weather_data_clouds[['Cloud9am','Cloud3pm']].isnull().sum() / weather_data_clouds.shape[0] * 100 

In [ ]:
weather_cloud_9am = weather_data_clouds['Cloud9am'].value_counts() 
weather_cloud_3pm = weather_data_clouds['Cloud3pm'].value_counts()
weather_cloud_9am.index = weather_cloud_9am.index.astype(int)
weather_cloud_3pm.index = weather_cloud_3pm.index.astype(int)
clouds = pd.DataFrame({'Clouds at 9am': weather_cloud_9am, 'Clouds at 3pm':weather_cloud_3pm})
ax = clouds.plot(kind='bar', figsize=(10, 6), width=0.5,color=['#DBDFEA', '#ACB1D6'], edgecolor='black')
ax.set_ylabel('Values')
ax.set_xlabel('Cloud cover')
ax.set_title('Cloudiness measurement at 9am and 3pm')
plt.legend(title='Time', fontsize=10)
plt.xticks(rotation=0)
sns.despine()
plt.show()

## Random Imputation

In [ ]:
# Cloud9am

In [ ]:
#Before
plt.figure(figsize=(8, 6))
sns.countplot(x='Cloud9am', data=weather_data_clouds)
plt.xlabel('Cloud9am')
plt.ylabel('Count')
plt.title('Cloud9am Value Counts')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# def fill_categorical_gaps(data,col):
#     if data[col].dtype == 'object':  # Check if the column is categorical
#         missing_indices = data[col].isnull()
#         non_missing_data = data.loc[~missing_indices, col]
#         probabilities = non_missing_data.value_counts(normalize=True)
#         num_missing = missing_indices.sum()
        
#         # Generate random values based on the probabilities
        
#         random_values = np.random.choice(probabilities.index, size=num_missing, p=probabilities.values)
#         data.loc[missing_indices, col] = random_values

#     return data

# fill_categorical_gaps(weather_data_clouds,'Cloud9am')

In [ ]:
missing_indices = weather_data_clouds['Cloud9am'].isnull()
num_missing = missing_indices.sum()
num_missing

In [ ]:
non_missing_data = weather_data_clouds.loc[~missing_indices,'Cloud9am' ]
non_missing_data.head(5)

In [ ]:
probabilities = non_missing_data.value_counts(normalize=True) # the relative frequencies (probabilities) instead of the actual counts.
random_values = np.random.choice(probabilities.index, size=num_missing, p=probabilities.values)
weather_data_clouds.loc[missing_indices,'Cloud9am'] = random_values
weather_data_clouds['Cloud9am'].value_counts()

In [ ]:
#After
plt.figure(figsize=(8, 6))
sns.countplot(x='Cloud9am', data=weather_data_clouds)
plt.xlabel('Cloud9am')
plt.ylabel('Count')
plt.title('Cloud9am Value Counts')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Cloud 3pm

In [ ]:
#Before
plt.figure(figsize=(8, 6))
sns.countplot(x='Cloud3pm', data=weather_data_clouds)
plt.xlabel('Cloud9am')
plt.ylabel('Count')
plt.title('Cloud9am Value Counts')
plt.xticks(rotation=45)
plt.show()

In [ ]:
missing_indices = weather_data_clouds['Cloud3pm'].isnull()
num_missing = missing_indices.sum()
non_missing_data = weather_data_clouds.loc[~missing_indices,'Cloud3pm']

probabilities = non_missing_data.value_counts(normalize=True) # the relative frequencies (probabilities) instead of the actual counts.
random_values = np.random.choice(probabilities.index, size=num_missing, p=probabilities.values)

weather_data_clouds.loc[missing_indices,'Cloud3pm'] = random_values

In [ ]:
#After
plt.figure(figsize=(8, 6))
sns.countplot(x='Cloud3pm', data=weather_data_clouds)
plt.xlabel('Cloud3pm')
plt.ylabel('Count')
plt.title('Cloud3pm Value Counts')
plt.xticks(rotation=45)
plt.show()

#### I have tried also with other methods,namely with the mode and RandomForestClassifier but I was not satisfied with the results. 

#### Ultimately, I chose filling NaNs with random values based on the distribution of the variable.

## Filling gaps with the mode

In [ ]:
# weather_data_clouds = weather_data_dir_with_mode.copy()
# clouds_9am_mode = weather_data_clouds['Cloud9am'].mode().iloc[0]
# clouds_3pm_mode = weather_data_clouds['Cloud3pm'].mode().iloc[0]

# weather_data_clouds['Cloud9am']= weather_data_clouds['Cloud9am'].fillna(clouds_9am_mode)
# weather_data_clouds['Cloud9am']= weather_data_clouds['Cloud9am'].fillna(clouds_9am_mode)

# weather_cloud_9am_with_mode = weather_data_clouds['Cloud9am'].value_counts()
# weather_cloud_3pm_with_mode = weather_data_clouds['Cloud3pm'].value_counts()

# clouds_with_mode = pd.DataFrame({'Clouds at 9am': weather_cloud_9am_with_mode, 'Clouds at 3pm':weather_cloud_3pm_with_mode})
# clouds_with_mode

# ax = clouds_with_mode.plot(kind='bar', figsize=(10, 6), width=0.5,color=['#DBDFEA', '#ACB1D6'], edgecolor='black')
# ax.set_ylabel('Values')
# ax.set_xlabel('Cloud cover')
# ax.set_title('Cloudiness measurement at 9am and 3pm \n after filling NaNs with mode')
# plt.legend(title='Time', fontsize=10)
# plt.xticks(rotation=0)
# sns.despine()
# plt.show()

# Random Forest Classifier
Random Forests are relatively resistant to null values and can handle missing data implicitly during the tree-building process.
RandomForestClassifier is a supervised learning algorithm - the model is trained on labeled data -> 
the input data (features) and their corresponding output labels (target) are provided during the training process.

Inputs (features) have to be converted to numeric - for categorical features I will use the target encoding (one-hot enocoding will cause too many classes)
Will it be good idea to combine target encoding with one-hot encoding? 

Outputs should remain categorical 

Target Leakage: "Ensure that the target encoding is calculated based only on the training set and not using any information from the validation or test sets."

Actually, from the very beginning I wanted to use RandomForestClassifier algorithm (method that consist of multiple Decision Trees), however it gave me less than 50% accuracy on test data and validation data. I tried to improve it by experimenting with: 
1) hyperparametrs like number of decision trees (n_estimators), maximum depth of trees(max_depth),minimum number of samples required to split a node (min_samples_split).
2) cross-validation 
3) class weights - it is recommended to set the class_weight parameter to "balanced" in RFC if there is class imbalance in the target variable
4) I was thinking about using one of the hyperparameter optimization techniques (e.g., grid search, random search, Bayesian optimization) to automatically search for the optimal number of trees, but for some reasons I could not run it. 
5) smoothing -> to enhence the target encoding process: to handle rare categories

I think **overfitting** is a major problem here but somehow I can't figure out the reason of it. I tried to optimalize both target encoding and RFC model and there goes something wrong. For now, I have no idea what. The code that I've tried is presented below.

In [ ]:
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

data_encoded = weather_data_clouds.copy()

# Conversion into categorical types (because in fact RainTomorrow, Cloud9am/3pm are categorical)
data_encoded['RainTomorrow'] = data_encoded['RainTomorrow'].map({0: 'No', 1: 'Yes'})
cloud_categories = {
    0: 'Clear',
    1: 'Few Clouds',
    2: 'Partly Cloudy',
    3: 'Mostly Cloudy',
    4: 'Cloudy',
    5: 'Overcast',
    6: 'Obscured',
    7: 'Mostly Obscured',
    8: 'Completely Overcast',
    9: 'Sky Obscured'
}
data_encoded['Cloud3pm'] = data_encoded['Cloud3pm'].map(cloud_categories).astype('category')

# Remove all rows with NaNs + remember to reset the indexes, later their might be problematic !!
data_encoded = data_encoded.dropna(how='any')
data_encoded.reset_index(drop=True, inplace=True)

target_column = 'Cloud9am'
categorical_columns_to_encode = ['Location', 'State/Province', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow', 'Cloud3pm']

X = data_encoded.drop(['Cloud9am'], axis=1)
y = data_encoded['Cloud9am']

# Scale features/inputs - without dummies !!

X_unscaled = X.drop(columns=categorical_columns_to_encode)
scaler = StandardScaler()
scaler.fit(X_unscaled)
X_scaled = scaler.transform(X_unscaled)
X_scaled = pd.DataFrame(X_scaled,columns=X_unscaled.columns)
X = pd.concat([X_scaled, X[categorical_columns_to_encode]], axis=1)

# Split the data into train,validation,train -> proportion: 70:10:20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

# Target encoding - watch that you fit only once, later you just use transform function on different data

for col in categorical_columns_to_encode:
    te = TargetEncoder(smoothing=0.2)
    te.fit(X_train[col], y_train)
    X_train[col] = te.transform(X_train[col])
    X_test[col] = te.transform(X_test[col])
    X_val[col] = te.transform(X_val[col])
    
# RandomForestClassifier

# rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
class_weights = 'balanced'

rf_classifier = RandomForestClassifier(class_weight = class_weights,max_depth = 40, min_samples_split = 5,random_state = 5)
rf_classifier.fit(X_train, y_train)

# Evaluate the model on the training set
training_accuracy = rf_classifier.score(X_train, y_train)
print("Training Accuracy:", training_accuracy)

validation_accuracy = rf_classifier.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

test_accuracy = rf_classifier.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)


## Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the random forest classifier
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5)  # 5-fold cross-validation
print("Cross-validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

## Learning curve 

In [ ]:
from sklearn.model_selection import learning_curve

# Create learning curve with the random forest classifier
train_sizes, train_scores, test_scores = learning_curve(
    rf_classifier, X_train, y_train, cv=5, train_sizes=np.linspace(0.1, 1.0, 5)
)

# Calculate mean and standard deviation of training and test scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label="Training Accuracy", color="blue")
plt.fill_between(
    train_sizes,
    train_mean - train_std,
    train_mean + train_std,
    alpha=0.1,
    color="blue",
)
plt.plot(train_sizes, test_mean, label="Validation Accuracy", color="red")
plt.fill_between(
    train_sizes,
    test_mean - test_std,
    test_mean + test_std,
    alpha=0.1,
    color="red",
)
plt.xlabel("Training Size")
plt.ylabel("Accuracy")
plt.title("Learning Curve")
plt.legend(loc="best")
plt.grid()
plt.show()

## Attempt to find the best hyperparameters 

In [ ]:
# Define a range of values for max_depth and min_samples_split to try
#max_depth_range = [10, 20, 30, None]  # None means no maximum depth, allowing trees to grow freely
max_depth_range = [10, 20, 30, None]
min_samples_split_range = [2, 5, 10, 20]

best_validation_accuracy = 0
best_max_depth = None
best_min_samples_split = None

for max_depth in max_depth_range:
    for min_samples_split in min_samples_split_range:
        rf_classifier = RandomForestClassifier(max_depth=max_depth, min_samples_split=min_samples_split, random_state=5)
        rf_classifier.fit(X_train, y_train)
        validation_accuracy = rf_classifier.score(X_val, y_val)
        print(f"Max Depth: {max_depth}, Min Samples Split: {min_samples_split}, Validation Accuracy: {validation_accuracy}")

## GridSearchCV

In [ ]:
# from sklearn.model_selection import GridSearchCV

# # Define the hyperparameter grid to search
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
# }

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # Perform the grid search on the training set
# grid_search.fit(X_train, y_train)

# # Get the best hyperparameters and model from the grid search
# best_rf_classifier = grid_search.best_estimator_

# # Evaluate the best model on the validation set
# validation_accuracy = best_rf_classifier.score(X_val, y_val)
# print("Best Validation Accuracy:", validation_accuracy)

## Feature importances

In [ ]:
rf_classifier.feature_importances_.shape

In [ ]:
X.columns

In [ ]:
features = X.columns
importances = rf_classifier.feature_importances_
sorted_indices = np.argsort(importances)[::-1] #sort indexes in descending order

plt.figure(figsize=(10, 6))
sns.barplot(x=importances[sorted_indices], y=[features[i] for i in sorted_indices])
plt.title('Feature Importances')
plt.xlabel('Relative Importance')
plt.ylabel('Features')
plt.show()

## Confusion matrix

A confusion matrix typically has four main components:

True Positive (TP): The number of instances that are correctly predicted as positive by the model.

False Positive (FP): The number of instances that are incorrectly predicted as positive by the model when they are actually negative.

True Negative (TN): The number of instances that are correctly predicted as negative by the model.

False Negative (FN): The number of instances that are incorrectly predicted as negative by the model when they are actually positive.

In [ ]:
confusion_matrix = pd.DataFrame({'Actual Positive': ['TP', 'FN'], 'Actual Negative': ['FP', 'TN']}, index=['Predicted Positive', 'Predicted Negative'])

print(confusion_matrix)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = rf_classifier.predict(X_test)
confusion_matrix(y_test,y_pred)
# It shows how many values are correct classified (those value are in the diagonal)

## Classification_report
Precision: The ability of the model to correctly identify instances of a given class. It is the ratio of true positive predictions to the total predicted positive instances.

Recall: The ability of the model to correctly identify all instances of a given class. It is the ratio of true positive predictions to the total actual positive instances.

F1-score: The harmonic mean of precision and recall, providing a balanced measure between precision and recall.

Support: The number of samples in each class.

Accuracy: The overall accuracy of the model, which is the ratio of correct predictions to the total number of samples.

Macro avg: The average of the metrics (precision, recall, F1-score) for all classes. It gives equal importance to each class.

Weighted avg: The weighted average of the metrics, taking into account the support (number of samples) for each class.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

## <span style='color: #3F2E3E;'>RainToday - nominal</span>
#### <span style='color: #A78295;'>If today is rainy then ‘Yes’. If today is not rainy then ‘No’.</span>

In [ ]:
# Checkpoint
weather_data_after_clouds = weather_data_clouds.copy()
weather_data_after_clouds['RainToday'].head(5)
weather_data_after_clouds['RainToday'].value_counts() #does not contain NaNs

In [ ]:
weather_data_after_clouds['RainToday'].isnull().sum()/weather_data_after_clouds.shape[0]

In [ ]:
#I'm going to remove all missing values, because they constitute only ~ 0.009%
weather_data_after_clouds = weather_data_after_clouds[weather_data_after_clouds['RainToday'].notnull()]
# 2nd way 
# weather_data_after_clouds= weather_data_after_clouds.dropna(subset=['RainToday'])
weather_data_after_clouds['RainToday'].isnull().sum()

In [ ]:
weather_data_after_clouds.shape

In [ ]:
sns.set(style="whitegrid", palette="pastel")

sns.countplot(x='RainToday', data=weather_data_after_clouds, hue='RainToday',  order=['Yes', 'No'],dodge=False)

plt.xlabel('Rain Today')
plt.ylabel('Count')
plt.title('Count of Rain Today')

plt.legend(title='Rain', loc='upper right', labels=['Yes', 'No'])

# Adjust figure size for better visualization
plt.figure(figsize=(6, 4))

plt.show()

In [ ]:
weather_data_after_clouds['RainToday'] = weather_data_after_clouds['RainToday'].map({'Yes':1,'No':0}) 
weather_data_after_clouds.head(10)

In [ ]:
weather_data_rain = weather_data_after_clouds.copy()

<h2 style='text-align: center;'> OUR MAIN PURPOSE </h2>

## <span style='color: #3F2E3E;'>RainTomorrow - nominal</span>
#### <span style='color: #A78295;'>If tomorrow is rainy then 1. If tomorrow is not rainy then 0 </span>

In [ ]:
weather_data_rain['RainTomorrow'].head(5)
weather_data_rain['RainTomorrow'].value_counts()

In [ ]:
weather_data_rain['RainToday'].value_counts() # Very similar to Rain Tomorrow -> Rain Today is a key variable for tomorrow rain prediction

In [ ]:
weather_data_rain['RainTomorrow'].isnull().sum()

In [ ]:
sns.set(style="whitegrid", palette="pastel")

sns.countplot(x='RainTomorrow', data=weather_data_rain, hue='RainTomorrow',dodge=False,order=[1,0])

plt.xlabel('Rain Today')
plt.ylabel('Count')
plt.title('Distribution of Rain Tomorrow')

plt.legend(title='Rain', loc='upper right', labels=['Yes', 'No'])

# Set custom labels for x-axis, position 0 -> yes, position 1 -> no 
plt.xticks(ticks=[0, 1], labels=['Yes', 'No'])

plt.figure(figsize=(6, 4))

plt.show()

In [ ]:
#Add correlation 

### The chi-square test - categorical vs. categorical variable
The Chi-square test is used to determine if there is a significant association between two categorical variables. 
Process for conducting a Chi-square test:
1. Formulate the null and alternative hypotheses:
Null hypothesis: There is no association between the two categorical variables.
Alternative hypothesis: There is a significant association - || - 
2. Set the significance level (alpha): Choose a significance level (commonly 0.05) that represents the threshold for determining statistical significance. The significance level determines how strong the evidence against the null hypothesis must be before we reject it.
3. Create a contingency table (cross-tabulation)- a table that shows the observed frequencies for each combination of the two categorical variables.
4. Compute the Chi-square test statistic - you calculate the difference between the observed and expected frequencies and then you sum up them.
5. Find p-value
6. Comparison: If the p-value is < 0.05, reject the null hypothesis - there is a significant association between the variables. If The p-value >= 0.05 reject alternative hypothesis - there is no significant association.
7. Chi-square statistic: The bigger difference, the stronger association between the variables. When the test statistic is large, it suggests that there is a significant discrepancy between the observed and expected frequencies, indicating that the variables are dependent.

#### I added also Cramer's V, which is an extension of the chi-square test and provides a standardized measure of association.
Cramer's V ranges from 0 to 1, where 0 indicates no association between the categorical variables, and 1 indicates a perfect association. The formula for Cramer's V is:

$$
V = \sqrt{\frac{\chi^2}{(n \cdot min (k-1,r-1)}}
$$

Where:

$\chi^2$  is the chi-square statistic obtained from the chi-square test of independence between the two categorical variables.

n is the total number of observations in the contingency table.

k is the number of rows in the contingency table.

r is the number of columns in the contingency table.

In [ ]:
# Problem 1) Cloud's type is by default float, so I need to change it:
cloud_categories = {
    0: 'Clear',
    1: 'Few Clouds',
    2: 'Partly Cloudy',
    3: 'Mostly Cloudy',
    4: 'Cloudy',
    5: 'Overcast',
    6: 'Obscured',
    7: 'Mostly Obscured',
    8: 'Completely Overcast',
    9: 'Sky Obscured'
}

cloud_9am = weather_data_clouds['Cloud9am'].map(cloud_categories).astype('category')
cloud_3pm =  weather_data_clouds['Cloud3pm'].map(cloud_categories).astype('category')

#other categorical variables
location = weather_data_clouds['Location']
state = weather_data_clouds['State/Province']
wind_gust = weather_data_clouds['WindGustDir']
wind_9am = weather_data_clouds['WindDir9am']
wind_3pm = weather_data_clouds['WindDir3pm']
rain_today = weather_data_clouds['RainToday']

# 'RainTomorrow' - 0 if will be raining 1 if not 
rain_tomorrow = weather_data_clouds['RainTomorrow'].map({0:'No',1:'Yes'})
# Create a crosstabulation between "cloud" and another categorical variable
#pd.crosstab(cloud_9am,cloud_3pm)
#pd.crosstab(cloud_9am,location)
#pd.crosstab(cloud_9am,state)
#pd.crosstab(cloud_9am,wind_gust)
#pd.crosstab(cloud_9am,wind_9am)
#pd.crosstab(cloud_9am,wind_3pm)
#pd.crosstab(cloud_9am,rain_today)
#pd.crosstab(cloud_9am,rain_tomorrow)

# We might assume that if the sky is completely overcast (2) or mostly obscured (5) there is a high probability that will be raining, 
# however "correlation does not imply causation". It means that just because two variables show a statistical relationship (correlation) does not necessarily mean that one variable causes the other to change.

pd.crosstab(cloud_3pm,rain_today)

# # Perform the chi-square test
location = weather_data_clouds['Location']
state = weather_data_clouds['State/Province']
wind_gust = weather_data_clouds['WindGustDir']
wind_9am = weather_data_clouds['WindDir9am']
wind_3am = weather_data_clouds['WindDir3pm']
rain_today = weather_data_clouds['RainToday']

categorical_tab = np.array([location,state,wind_gust,wind_9am,wind_3am,cloud_9am,cloud_3pm,rain_today,rain_tomorrow])
categorical_tab_str = np.array(['location','state','wind_gust','wind_9am','wind_3am','cloud_9am','cloud_3pm','rain_today','rain_tomorrow'])

def chi_square_test(categorical_var_1, categorical_var_2):
    crosstab = pd.crosstab(categorical_var_1, categorical_var_2)
    chi2, p, _, _ = chi2_contingency(crosstab)
    n = crosstab.sum().sum()
    # crosstab.sum() - the sum of elements along each column of the contingency table
    # crosstab.sum().sum() - the sum of all elements in the 1-dimensional array obtained from the previous step
    num_rows = crosstab.shape[0]
    num_cols = crosstab.shape[1]
    cramer_v = np.sqrt(chi2 / (n * min(num_rows - 1, num_cols - 1)))
    return chi2, p, cramer_v

pd.crosstab(cloud_9am, cloud_3pm)
chi_square_test(cloud_9am, cloud_3pm)

def chi_square_contigency(tab_x, tab_x_str, var):
    df = pd.DataFrame(columns=['Variable', 'Chi-square', 'P-value',"Cramer's V"])
    for x, x_str in zip(tab_x, tab_x_str):
        chi2, p, cramer_v = chi_square_test(var, x)
        result = pd.DataFrame({'Variable': [x_str], 'Chi-square': [round(chi2,2)], 'P-value': [p], "Cramer's V":[round(cramer_v,2)]})
        df = pd.concat([df, result], ignore_index=True)
        df = df.sort_values("Cramer's V",ascending=False)
    return df

chi_square_contigency(categorical_tab,categorical_tab_str,cloud_9am)
chi_square_contigency(categorical_tab,categorical_tab_str,cloud_3pm)


**In summary**, the results show that all the variables (cloud_3pm, location, rain_tomorrow, rain_today, state, wind_9am, wind_gust, wind_3am) have a significant association with the variable "cloud_9am.". 
The same with "cloud_3pm", large chi-squares and small p-values (close to 0) indicate strong evidence against the null hypothesis and prove that there is a significant association between "cloud_3pm" and other variables. Moreover,The strength of association varies, with some variables exhibiting stronger associations (Cramer's V close to 1) and others having weaker associations (Cramer's V closer to 0). 

# To measure the dependency between a categorical variable and a numerical variable, we may use varoius methods:
1. ANOVA (Analysis of Variance)-  for differences in means between multiple groups of a categorical variable with respect to a numerical variable.
It assesses whether there is a statistically significant difference in the means of the numerical variable across the different categories.
However, ANOVA assumes that the numerical variable follows a normal distribution within each group. 

2. Kruskal-Wallis Test: is a non-parametric test that can be used when the assumption of normality is violated (is suitable when the numerical variable is not normally distributed.
It tests whether the median of the numerical variable differs significantly across the categories of the categorical variable.
                                                                                                                
3. Point-Biserial Correlation / Biserial Correlation: if the categorical variable is binary (only two classes)
There are also other options but those are the most popular.  
                                                                                                                
I decided to use Kruskal-Wallis Test because it is safe option here. 
                                                                                                  

In [ ]:
# https://colorhunt.co/palettes/vintage